# WHIN Supply Chain Web Scraper 

### Team Members 
1. Deepika Jindal 
2. Mihir Bhatia 

## Importing Packages 

In [12]:
import urllib.request
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urltools
import time 
from pandas import ExcelWriter
from pandas import ExcelFile
import time

## Get base URL of the website 

This function lets you extract the base URL of the website. It will be used to check if a web page is from the original webiste that we intended to scrape. 

In [13]:
def get_base_url(url):
    from urllib.parse import urlsplit
    base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url))
    

    if base_url.startswith("https"):
        base_url = base_url.replace('https://','')
    elif base_url.startswith("http"):
        base_url = base_url.replace('http://','')
        
    if base_url.endswith('/'):
        base_url = base_url.replace('/','')
    return base_url


##  Does the page exist ? 
Our web scrapper will go the website and use the http protocol to check if the web page which we intend to scrape exists 

In [14]:
def does_page_exist(url):
    exists = 0 
    try:
        request = requests.get(url)
        if request.status_code == 200:
            exists = 1 
            
    except: 
        pass
    
    return exists



In [15]:
# Function to extract links from a URL------------------------------------------
def href_scrapper(url):

    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        a_tag = soup.find_all('a')
        img_tag = soup.find_all('img')
        link_tag = soup.find_all('link')
        link_list = list()

        for links in a_tag:
            link_list.append(links.get('href'))

        for links in link_tag:
            link_list.append(links.get('href'))

        for links in img_tag:
            link_list.append(links.get('src'))

        return link_list

    except:
        return 0

## What if the link extracted has no http in it ? 
There will be cases in which the link you extracted has no http in the beginning. This function adds an http to the beginning of the link to keep all links similar in structure. This will be useful during string manipulations and will standardize link structure 

In [16]:
# Function to create complete links---------------------------------------------
def link_former(url):
    x = url

    if x.startswith("http"):
        return x

    else:
        full_url = original_url + x
        return full_url



## Save files 
This function saves files with various formats. Our main web scrapper doesn't use all these extensions. But we provide this function incase somoene wants to scrape images, PDFs and webpages 

In [17]:
# Function to save links to directory of python file----------------------------
def file_save(url, i):
    save_error = 0
    full_url = url
    try:
        if str(full_url).endswith(".htm"):
            name = "WebPage" + str(i) + ".htm"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".pdf"):
            name = "Pdf" + str(i) + ".pdf"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".jpg"):
            name = "Image" + str(i) + ".jpg"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".jpeg"):
            name = "Image" + str(i) + ".jpeg"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".png"):
            name = "Image" + str(i) + ".png"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        elif str(full_url).endswith(".css"):
            name = "CSS File" + str(i) + ".css"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        else:
            name = "WebPage" + str(i) + ".html"
            urllib.request.urlretrieve(full_url, name)
            m = i + 1

        return m
    except:
        save_error = save_error + 1



## Now it's time to crawl
The crawl function takes a link, with a number ID and output file and outputs all the daughter links associated with that parent link. 

![Layered stucture of Crawled Websites](crawl_structure.png)

In [18]:
'''
crawl function = takes link as input and crawls relevant webpages from the parent 

Inputs 
----------

Outputs
----------
'''
def crawl(original_url, num_Id, output_file):
    
    # Accepting input of URL and depth----------------------------------------------
    tic = time.time()
    
    parent_list = [original_url]
    url_to_check = get_base_url(original_url)
    print(url_to_check)
    url_to_check = str(url_to_check)

    layer_stop = 1
    layer = 0


    #initializing all the requiered lists---------------------------------------------
    visited_all  = [original_url]
    visited_current_layer = []
    child_list =[]
    child_list_filtered = [original_url]
    #columns = ['Link','Parent Link', 'Layer']
    df = pd.DataFrame()

    Di={}

    # Main execution of scrapper----------------------------------------------------

    #looping through layers 
    while layer < layer_stop:
    
        #looping through URLs in parent-list
        for url in parent_list:
        
            #scraping the children from the parent url----------------------------
            if href_scrapper(url) != 0:
                child_list = href_scrapper(url)
    
            
            for child in child_list:
                if child != None:
                    ch=child
                    #if child link is of the form "index.php/blahblah" and parent ends with '/'
                    #---> "parentlink/index.php/blahblah"
                    if child.startswith('/'):
                        child= str(url) + str(child)
                    
                    if url.endswith('/') and url_to_check not in child:
                        child = str(url) + str(child)
                    
                #normalize the child links-------------------------------------
                    child=urltools.normalize(child)  
                    

                    #filtering out based on 1) External 2) Repeating 3) Invalid links---------------------------
                    if url_to_check in child and child not in visited_all and does_page_exist(child)==1 and ch not in Di:
                        child_list_filtered.append(child)
                        Di[ch]=1
                
                    #adding everthing to visited all--------------------
                    if child not in visited_all:
                        child_slash = child + '/'
                        visited_all.append(child)  
                        visited_all.append(child_slash)
                    
                    
            #adding  the visited and filtered children into the "current visited layer" ------ 
            for child_filtered in child_list_filtered:
                visited_current_layer.append(child_filtered)

            #creating a Pandas dataframe to store everything for download----------
            layer_number = [layer+1]*len(child_list_filtered)
            parent_of_child = [url]*len(child_list_filtered)

            df_child = pd.DataFrame(child_list_filtered)
            df_parent = pd.DataFrame(parent_of_child)
            df_layer = pd.DataFrame(layer_number)


            df_to_be_added = pd.concat([df_child,df_parent,df_layer], axis=1)
            df = pd.concat([df,df_to_be_added],ignore_index=True, axis = 0)
            #----------------------------------------------------------------------
        
            #emptying the child lists
            child_list = []
            child_list_filtered = []
        
        #condition to stop filtering-----------------------------------------------
        if not visited_current_layer :
            layer_stop = layer_stop 
        else:
            layer_stop += 1

    
        #child layer is now parent layer--------------------------------------------
        parent_list = []
    
        #we only dont add .png, .jpg , .pdf to the new parent layer 
        for visited_current in visited_current_layer: 
            print(visited_current)
            if(not visited_current.endswith('.png') and not visited_current.endswith('.jpg') and not  visited_current.endswith('.pdf')):
                parent_list.append(visited_current)
            
            
        #displaying the links in different layers----------------------------------
        #print("Links in LAYER:" + str(layer+1))
        print("No of links = " + str(len(visited_current_layer)))
        #print(visited_current_layer)
        print("\n")
        visited_current_layer = [] 
        #updating the layer number
        layer +=1
    df.to_csv(output_file + '/' + str(num_Id) + '_' + str(url_to_check) +  '.csv', sep=',', encoding='utf-8')
    return df, num_Id

## Functions that will be used to scrape the crawled webpages 

In [19]:
'''
Functions that will be used by the scraper function 
tag_visible = 
text_from_html = 
'''

import requests
page = requests.get("https://docs.python.org/3/howto/urllib2.html")
from bs4 import BeautifulSoup
from bs4.element import Comment
soup = BeautifulSoup(page.content, 'html.parser')


#information based on:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True
p_tag=soup.find_all('p')

#extract text from html: 
def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)
web_text = []

## Scrape the webpages 
This function allows us to scrape all the webpages inside the dataframe that contains the crawled links

In [33]:
import json
import urllib
import os 
extensions=('.png', '.jpg','.jpeg','.pdf', '.gif','.ico')
def scrape(df, scraped_website_path, website_index, website_name):
    TempD={}
    for index,item in df.iterrows():
        #print(item[1],item[0])
        try:      
            #for i in extensions:
            #print(item[1])
            if item[1].endswith(extensions):
                print(" ")
                print(item[1],"skipped because link contains unwanted extension")
                
            elif ".gif&" in item[1]:
                print(" ")
                print(item[1]," skipped because of .gif in link")
                
                
            else:
                print(" ")
                print(item[1])
                html = urllib.request.urlopen(item[1]).read()
                #web_text.append(text_from_html(html))
                print(text_from_html(html))
                time.sleep(3)
                #save_url(str(item[1]), index)
                TempD[item[1]]=text_from_html(html)
        #label: end
        except: 
            pass
    with open(scraped_website_path + '/' + 'json_' + str(website_index) + str(get_base_url(website_name)) +  '.json', 'w') as outfile:  
        json.dump(TempD, outfile)
        
    return 1

## Crawler - that works with an excel sheet containing all the relevant webpages 

In [26]:

# web_crawler_all() = Scraper for multiple websites# web_cr 
'''
Inputs
-----------
excel_path = Excel file path which contains list of websites to be crawled  
sheet = sheet name of this excel file 
web_col = the column no of in which the website hompages are located 
index_col = the column with indices of websites
parent_col = the column with the parent names
output_folder = path of the output folder where we wish to save the websites 
from_website = the index no. from website (typically starting from 1)
to_website = the index no of the website till we want to continue scraping (scraped websites will include the 
                websites with this index)

Output 
-----------
CSV files containing a list of child links for each website are saved into the output_folder path 

'''
def web_crawler_all(excel_path, sheet, web_col, index_col, parent_col,  output_folder, from_website, to_website):
    df = pd.read_excel(excel_path , sheet_name=sheet)
    df = df.iloc[from_website-1 : to_website, :]
    #print(df)
    for index,row in df.iterrows():
        num_Id=int(row[index_col])
        original_url=row[web_col]

        
        #try:
        #websites= pd.read_csv(output_folder + str(int(num_Id)) + '_' + str(original_url) + '.csv')
        #print("website is already crawled")
            
        #except:
        websites, num_Id = crawl(original_url ,num_Id ,output_folder)
            
        
    print("All links from the websites indicated have been crawled")
    print("Output File :  " + str(output_folder))
    print("No. of websites scraped : " + str(to_website - from_website + 1))
    return 1

## Scraper that works with a file containing all the scrapped webpages in an excel sheet

In [32]:
'''
Inputs 
----------
excel_path = Excel file path which contains list of websites to be crawled  
sheet = sheet name of this excel file 
crawled_website_path = path of file where all the crawled websites are stored  
scraped_website_path = path of file where all the scraped websites are stored
from_website - to_website = website index number from-to which we want to scrape and store in JSON 
web_col = column number of websites (0,1,2....)

Output
----------
JSON files for each parent website in scraped_website_path folder. The path of the scraped websites 
is the same as that crawled websites 
'''

def web_scraper_all(df, crawled_website_path, scraped_website_path, from_website, to_website,web_col):
    #get excel sheet containing websites 
  #  df = pd.read_excel(excel_path , sheet_name=sheet)
    df = df.iloc[from_website-1 : to_website, :]   
    
    #loop over all the websites and extract child links. Store them in JSON files 
    
    for index, rows in df.iterrows():
        #setting the website index and name 
        website_index = int(rows[0] )
        website_name = rows[web_col]
        
        #generating filename and getting the csv file of child links 
        filename = crawled_website_path + '/'+ str(website_index)+ '_' + str(get_base_url(website_name)) + ".csv"
        df_of_child_links = pd.read_csv(filename)
        #print(filename,df_of_child_links )
        #executing the scrape function 
        flag = scrape(df_of_child_links , scraped_website_path, website_index, website_name) 
        
        #checkpoint to see if scraping is complete 
        if flag == 1 :
            print(website_name + ' has been scraped successfully')
            print(' ')
            #print('scraping percentage complete = ' + str((i/to_website)*100) + '%')
            flag = 0

## Excection of the crawler_all function 

In [29]:
#executing the crawler function 

excel_path = 'batch2.xlsx'
sheet = 'Sheet1'
web_col = 2
index_col = 0
parent_col = 1
output_folder = 'crawl_trial'
from_website = 1
to_website = 1

web_crawler_all(excel_path,sheet, web_col, index_col, parent_col,output_folder, from_website, to_website)

www.tmfcenter.com
https://www.tmfcenter.com
https://www.tmfcenter.com/
https://www.tmfcenter.com/history
https://www.tmfcenter.com/steel
https://www.tmfcenter.com/cnc-machining
https://www.tmfcenter.com/fabrication
https://www.tmfcenter.com/quality
https://www.tmfcenter.com/employment
https://www.tmfcenter.com/contact-1
https://www.tmfcenter.com/contact
No of links = 10


https://www.tmfcenter.com/verticle
https://www.tmfcenter.com/horizontal
https://www.tmfcenter.com/turning
https://www.tmfcenter.com/high-speed-drill-tap
https://www.tmfcenter.com/5-axis-machining
https://www.tmfcenter.com/custom-shaped-bar
https://www.tmfcenter.com/sawing
https://www.tmfcenter.com/forming
https://www.tmfcenter.com/welding
https://www.tmfcenter.com/painting
https://www.tmfcenter.com/roll-forming
https://www.tmfcenter.com/robitic-welding
https://www.tmfcenter.com/shot-blasting
https://www.tmfcenter.com/iso-certified
https://www.tmfcenter.com/advanced-quality-system
https://www.tmfcenter.com/low-ppm
http

1

## Execution of the scraper_all function 

In [34]:
#executing the scraper function 

excel_path = 'batch2.xlsx'
sheet = 'Sheet1'
df = pd.read_excel(excel_path , sheet_name=sheet)
crawled_website_path = 'crawl_trial'
scraped_website_path = 'scraped_trial'
from_website = 1
to_website = 1
web_col = 2 
df = pd.read_excel(excel_path , sheet_name=sheet)
#print(df)
web_scraper_all(df, crawled_website_path, scraped_website_path, from_website, to_website,web_col)

 
https://www.tmfcenter.com
    TMF  Home About Us Steel CNC Machining Fabrication Quality Employment Contact More Center  CONTACT US  ​  Address: TMF CENTER Phone: 765-762-1000  300 W. Washington Street  Williamsport, IN 47993   © 2016 TMF Center. Proudly created with Wix.com  TMF Center will provide our customers with high quality, competitively priced products that are manufactured consistently on time while continually improving our quality management system.            Not risk-adverse to purchasing mill buys of special material  ​  A Creative approach to new business and processing       Financially positioned to compete  ​  Quality, Delivery, Cost and Creativity            A Highly motivated work force  ​  ​  A strong customer focused company culture          
 
https://www.tmfcenter.com/
    TMF  Home About Us Steel CNC Machining Fabrication Quality Employment Contact More Center  CONTACT US  ​  Address: TMF CENTER Phone: 765-762-1000  300 W. Washington Street  Williamsport, IN

 
https://www.tmfcenter.com/contact-1
    TMF  Home About Us Steel CNC Machining Fabrication Quality Employment Contact More Center  CONTACT US  ​  Address: TMF CENTER Phone: 765-762-1000  300 W. Washington Street  Williamsport, IN 47993   © 2016 TMF Center. Proudly created with Wix.com  Contact Address: TMF Center  300 W Washington Street  Williamsport IN, 47993  ​  Phone: 765-762-1000 Send         
 
https://www.tmfcenter.com/contact
    TMF  Home About Us Steel CNC Machining Fabrication Quality Employment Contact More Center  CONTACT US  ​  Address: TMF CENTER Phone: 765-762-1000  300 W. Washington Street  Williamsport, IN 47993   © 2016 TMF Center. Proudly created with Wix.com     
 
https://www.tmfcenter.com/verticle
    TMF  Home About Us Steel CNC Machining Fabrication Quality Employment Contact More Center  CONTACT US  ​  Address: TMF CENTER Phone: 765-762-1000  300 W. Washington Street  Williamsport, IN 47993   © 2016 TMF Center. Proudly created with Wix.com  Vertical CNC Mach

 
https://www.tmfcenter.com/iso-certified
    TMF  Home About Us Steel CNC Machining Fabrication Quality Employment Contact More Center  CONTACT US  ​  Address: TMF CENTER Phone: 765-762-1000  300 W. Washington Street  Williamsport, IN 47993   © 2016 TMF Center. Proudly created with Wix.com  ISO Certified  TMF Centers Quality Management System has been ISO Certified since October 2006. Our current ISO Certification is to the ISO 9001:2008 Standard.            
 
https://www.tmfcenter.com/advanced-quality-system
    TMF  Home About Us Steel CNC Machining Fabrication Quality Employment Contact More Center  CONTACT US  ​  Address: TMF CENTER Phone: 765-762-1000  300 W. Washington Street  Williamsport, IN 47993   © 2016 TMF Center. Proudly created with Wix.com  Advanced Quality Systems  TMF incorporates various tools to ensure Quality is a key focus for both standard processes as well as process improvement.  •APQP  •Standard Work  •Statistical Process Control (CPK Analysis)  •8D Correctiv